In [1]:
## Refugia Project

In [2]:
#### Preprocessing steps for data can be done in python as well but I used data I had in ArcGis:
"""  
1) Download fire perimeters from mtbs.gov
The Monitoring Trends in Burn Severity Dataset has accuracy advantages over other 
fire perimter datasets see Kolden and Weisberg's (2007) in Fire Ecology

2) Clip fires by geographic boundry.
The refugia dataset includes all portions of any fire within Idaho, Oregon and Washington.
It does not include any information on the portions of the fires which lie outside of those states.

3) Merge 
"""

"  \n1) Download fire perimeters from mtbs.gov\nThe Monitoring Trends in Burn Severity Dataset has accuracy advantages over other \nfire perimter datasets see Kolden and Weisberg's (2007) in Fire Ecology\n\n2) Clip fires by geographic boundry.\nThe refugia dataset includes all portions of any fire within Idaho, Oregon and Washington.\nIt does not include any information on the portions of the fires which lie outside of those states.\n\n3) Merge \n"

In [3]:
import pandas as pd
import numpy as np
import os
import glob
import geopandas
from shapely.geometry import Point, Polygon

import matplotlib.pyplot as plt
import time
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_ind



%matplotlib inline
width = 20
height = 20
plt.rcParams['figure.figsize'] = [width, height]

pd.options.display.max_columns = None

## Check if geopandas is installed
## import sys
## 'geopandas' in sys.modules


## Set Directory
path = '/data/yoder/RefugiaProject/'
os.chdir(path)
# os.listdir()

In [4]:
## Read in fire perimeter files
# MTBS = geopandas.read_file('/data/yoder/DensityProject/mtbs_perimeter_data/mtbs_perims_DD.shp')
# tosave = MTBS.crs

## Load fires completely contained in PNW so refugia stats are correct
## This is the list of fires completely contained (described above)
# touse = pd.read_csv('/data/yoder/RefugiaSample.csv')

In [5]:
## Load fire data and set to a projected coordinate system

# Sample = touse[['Fire_ID', 'MTBS_Name']].copy()
# touse = touse.drop(['Unnamed: 0'],axis=1)
# touse = touse.merge(MTBS,how='left',on='Fire_ID')
# Fires = geopandas.GeoDataFrame(touse, geometry='geometry')
# Fires.crs = MTBS.crs
# Fires = Fires.to_crs("EPSG:2163")

In [6]:
## Load (or save) the projected fires

# Fires.to_file('/data/yoder/RefugiaProject/FiresEPSG2163.shp')
Fires = geopandas.read_file('/data/yoder/RefugiaProject/FiresEPSG2163.shp')
Flist = Fires['Fire_ID'].copy()

In [7]:
## Load (or save) projected refugia for the clipped fires above

# Refugia = geopandas.read_file('/data/yoder/RefugiaProject/RefugiaZillow/Fires_and_Refugia/RefugiaContained.shp')
# Refugia = Refugia.to_crs("EPSG:2163")
# Refugia.to_file('/data/yoder/RefugiaProject/Refugia2163.shp')
Refugia = geopandas.read_file('/data/yoder/RefugiaProject/Refugia2163.shp')

In [8]:
## %%time
## Dissolve rufugia (create a sinlge row of data with for all refugia patches within each fire) 
## Takes around 27 minutes to

# ref = Refugia.dissolve(by='fire_id')

In [9]:
## Save or load single refugia data

# ref.to_file('/data/yoder/RefugiaProject/Refugia_dissolved.shp')
ref = geopandas.read_file('/data/yoder/RefugiaProject/Refugia_dissolved.shp')

In [10]:
## Select refugia which are within our list of fires
## Update crs after using pandas merge (the geodataframe is lost when using pandas commands )

R = geopandas.pd.merge(Flist, ref, how='inner', left_on='Fire_ID',right_on='fire_id')
R = geopandas.GeoDataFrame(R, geometry = R['geometry'])
R.crs = ref.crs

In [11]:
## Create list of refugia 
Rlist = R['fire_id']

In [12]:
## Select refugia (not dissolved) that are within our fires
## Result has 621951 rows and 6 columns

Ref = geopandas.pd.merge(Rlist, Refugia, how='inner')
Ref = geopandas.GeoDataFrame(Ref, geometry=Ref['geometry'])
Ref.crs = R.crs

In [13]:
## Select fires which for which we have refugia data

F = geopandas.pd.merge(Fires, ref, how='inner', left_on='Fire_ID',right_on='fire_id')
F = geopandas.GeoDataFrame(F, geometry = F['geometry_x'])
F.crs = Fires.crs

In [28]:
F['area(ha)'].describe()

count    2053.000000
mean       44.318434
std       115.799495
min         0.000000
25%         7.135200
50%        13.367700
75%        34.456500
max      2255.490000
Name: area(ha), dtype: float64

In [14]:
######## Load and summarize Alexandre et al. Data
## Available at:  https://datadryad.org/stash/dataset/doi:10.5061/dryad.h1v2g

In [15]:
# Alex = geopandas.read_file('/data/yoder/RefugiaProject/RefugiaZillow/Alexandre/PNW_Alexandre.shp')
# Alex = Alex.to_crs("EPSG:2163")
# Alex.to_file('/data/yoder/RefugiaProject/AlexandreData2163.shp')
Alex = geopandas.read_file('/data/yoder/RefugiaProject/AlexandreData2163.shp')

In [16]:
## Join to our fires
A_join = geopandas.pd.merge(Alex, F, how='inner',on='Fire_ID', suffixes=['_a','_f'])

In [17]:
## Create valid geometry columns within the Alexandre data to enable easy switching from one to the other
A_join['geometry'] = A_join['geometry_f'].copy()
A_join['geometry_f'] = A_join['geometry'].copy()
A_join['geometry'] = A_join['geometry_a'].copy()

In [18]:
## Define distance function to check if datapoint is outside of fire perimeter

def get_distance(row):
    distance = row.geometry.distance(row.geometry_f)
    return distance

In [19]:
A_join['Dist_perim'] = A_join.apply(lambda row: get_distance(row), axis=1)

In [20]:
## Select only Alexandre points that are correctly attributed to a fire in our sample
## Create different dataframes to describe structure outcomes by location

## Fires in perimeter (not necessarly in refugia) 
## Alex_in has 4865 rows and 73 columns (most columns are unnecessary as they are repeated elsewhere)
Alex_in = A_join.loc[(A_join['Dist_perim'] == 0)].copy() 

## Fires outside of fire perimter
Alex_outPerim = A_join.loc[(A_join['Dist_perim'] > 0)].copy()

## Fires within perimter and within refugia 1045 rows and 79 columns (again, most columns are unnecessary as they are repeated elsewhere)
Alex_refugia = geopandas.sjoin(Alex_in, Ref, how='inner', op='intersects')
Alex_ref_matches = Alex_refugia.loc[(Alex_refugia['Fire_ID'] == Alex_refugia['fire_id_right'])]
Alex_refugia = Alex_ref_matches.copy()

In [21]:
Alex_in

,Name,PopupInfo,YEAR,TYPE_H,Area,Perimeter,Acres_a,Fire_ID,Fire_Name_a,StartMonth_a,StartDay_a,Confidence,Comment,FireType,Distance,STATEABREV,NA_L2CODE,NA_L2NAME,NA_L1CODE,NA_L1NAME,NA_L2KEY,NA_L1KEY,geometry_a,ValidMTBS,MTBS_Name,Fire_NO,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name_f,Year,StartMonth_f,StartDay_f,Fire_Type,Acres_f,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry_f,geometry,Dist_perim
0,UK_2000_037,1998;2002;2002;3,2000,UK,6.436686e+06,18276.20788,1590.539934,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275523.152 -63536.007 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.4854,0.1701,2.62281,0.00000,0.527338,12.1010,87.3161,0.013877,0.041632,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.0,1,0.0,0.0,0.0,0.0,6.0,0.044076,0.054027,0.0,0.005772,MP 77 I-84,2000,7,17,Wildfire,1591.0,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.0,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275523.152 -63536.007 0.000),0.0
1,UK_2000_037,1998;2002;2002;3,2000,UK,6.436686e+06,18276.20788,1590.539934,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275291.851 -63337.757 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.4854,0.1701,2.62281,0.00000,0.527338,12.1010,87.3161,0.013877,0.041632,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.0,1,0.0,0.0,0.0,0.0,6.0,0.044076,0.054027,0.0,0.005772,MP 77 I-84,2000,7,17,Wildfire,1591.0,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.0,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275291.851 -63337.757 0.000),0.0
2,UK_2000_037,1998;2002;2002;3,2000,UK,6.436686e+06,18276.20788,1590.539934,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275320.547 -64011.052 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.4854,0.1701,2.62281,0.00000,0.527338,12.1010,87.3161,0.013877,0.041632,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.0,1,0.0,0.0,0.0,0.0,6.0,0.044076,0.054027,0.0,0.005772,MP 77 I-84,2000,7,17,Wildfire,1591.0,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.0,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275320.547 -64011.052 0.000),0.0
6,UH_2000_039,1998;2003;2003;3,2000,UH,1.277779e+08,71253.13990,31574.598478,ID4320511555120000807,OREGON TRAIL,8,7,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1254228.870 -80538.673 0.000),ID4320511555120000807,OREGON TRAIL,41,2000220,41,30,11,127.8330,4.5135,3.53077,3.85533,0.421721,63.4947,34.7079,0.397080,0.975098,0.00352,LE70410302000218EDC00,LT50410302001228LGS01,LE70410301999231EDC00,LT50410302000226XXX03,0,0,0.0,1,0.0,0.0,0.0,22.0,28.0,0.033368,0.047847,0.0,0.006218,OREGON TRAIL,2000,8,7,Wildfire,31575.0,"POLYGON ((-1259155.779 -72534.356, -1259136.65...",ID4320511555120000807,120490,41,41,3600.0,MULTIPOLYGON Z (((-1246279.909 -89925.884 0.00...,"POLYGON ((-1259155.779 -72534.356, -1259136.65...",POINT Z (-1254228.870 -80538.673 0.000),0.0
7,UH_2000_039,1998;2003;2003;3,2000,UH,1.277779e+08,71253.13990,31574.598478,ID4320511555

In [23]:
Alex_in.head(1)

,Name,PopupInfo,YEAR,TYPE_H,Area,Perimeter,Acres_a,Fire_ID,Fire_Name_a,StartMonth_a,StartDay_a,Confidence,Comment,FireType,Distance,STATEABREV,NA_L2CODE,NA_L2NAME,NA_L1CODE,NA_L1NAME,NA_L2KEY,NA_L1KEY,geometry_a,ValidMTBS,MTBS_Name,Fire_NO,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name_f,Year,StartMonth_f,StartDay_f,Fire_Type,Acres_f,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry_f,geometry,Dist_perim
0,UK_2000_037,1998;2002;2002;3,2000,UK,6.436686e+06,18276.20788,1590.539934,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275523.152 -63536.007 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.4854,0.1701,2.62281,0.0,0.527338,12.101,87.3161,0.013877,0.041632,0.0,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.0,1,0.0,0.0,0.0,0.0,6.0,0.044076,0.054027,0.0,0.005772,MP 77 I-84,2000,7,17,Wildfire,1591.0,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.0,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275523.152 -63536.007 0.000),0.0


In [793]:
## Outcomes for fires outside of perimeter
Alex_outPerim['TYPE_H'].value_counts()

UH    1849
NH      63
BH      18
UK      17
RH       4
Name: TYPE_H, dtype: int64

In [794]:
## Outcomes for fires inside of perimeter (not necessarily in refugia)
Alex_in['TYPE_H'].value_counts()

UH    3935
BH     434
UK     256
NH     195
RH      45
Name: TYPE_H, dtype: int64

In [256]:
## Outcomes for fires inside of perimeter AND in refugia 
Alex_refugia['TYPE_H'].value_counts()

UH    942
UK     47
NH     34
BH     22
Name: TYPE_H, dtype: int64

In [771]:
## Create dataframes of Unburned, Burned and Rebuilt houses within perimter

Alex_in_UH =Alex_in[Alex_in['TYPE_H']=='UH'].copy()
Alex_in_BH =Alex_in[Alex_in['TYPE_H']=='BH'].copy()
Alex_in_RH =Alex_in[Alex_in['TYPE_H']=='RH'].copy()

In [239]:
## Create dataframes of Unburned, Burned and Rebuilt houses within refugia

Alex_ref_UH =Alex_refugia[Alex_refugia['TYPE_H']=='UH'].copy()
Alex_ref_BH =Alex_refugia[Alex_refugia['TYPE_H']=='BH'].copy()
Alex_ref_RH =Alex_refugia[Alex_refugia['TYPE_H']=='RH'].copy() # There aren't any of these

In [242]:
## Summarize the area of the refugia patches that a UNBURNED house resides in
Alex_ref_UH['F_AREA_left'].describe()

count       942.000000
mean       7437.898089
std       25197.014082
min        1800.000000
25%        1800.000000
50%        2700.000000
75%        6300.000000
max      322200.000000
Name: F_AREA_left, dtype: float64

In [243]:
## Summarize the area of the refugia patches that a BURNED house resides in
Alex_ref_BH['F_AREA_left'].describe()

count        22.000000
mean      17509.090909
std       68096.919090
min        1800.000000
25%        1800.000000
50%        1800.000000
75%        4050.000000
max      322200.000000
Name: F_AREA_left, dtype: float64

In [245]:
## Test difference in areas burned and unburned
ttest_ind(Alex_ref_UH['F_AREA_left'], Alex_ref_BH['F_AREA_left'])

Ttest_indResult(statistic=-1.737532975547313, pvalue=0.08261316748865234)

In [45]:
## ADD IN MICROSOFT DATA

In [32]:
%%time
### Read in ID, WA, OR Microsoft Building Footprints Data takes 5-6 min
""" 
The data here contains LANDFIRE burn probability and conditional flame length
data for each structures but those fields aren't used in this paper
You can download the file here https://github.com/Microsoft/USBuildingFootprints
It comes as relatively large GEOJSON files that can be hard to manage on
a local machine.
Some of the Microsoft files will kept crashing a jupyter notebook
I have better luck using Spyder when loading the files on a 
single machine (I have no idea why). 

"""

OR = geopandas.read_file('/data/yoder/StructureswithRisk/OR_points.shp')
ID = geopandas.read_file('/data/yoder/StructureswithRisk/ID_points.shp')
WA = geopandas.read_file('/data/yoder/StructureswithRisk/WA_points.shp')

CPU times: user 5min 22s, sys: 7.94 s, total: 5min 30s
Wall time: 5min 32s


In [44]:
%%time
## Convert CRS to match our data takes 5-6 minutes

OR = OR.to_crs(Fires.crs)
ID = ID.to_crs(Fires.crs)
WA = WA.to_crs(Fires.crs)

CPU times: user 5min 28s, sys: 3.94 s, total: 5min 32s
Wall time: 5min 33s


In [269]:
%%time
## Spatial join to get structures for all three states.
## this is very fast as it uses the spatial index takes < 30 seconds
## 18760 rows, 61 columns (lots of redundant columns)

MS_WA = geopandas.sjoin(F, WA, how='inner', op='intersects')
MS_ID = geopandas.sjoin(F, ID, how='inner', op='intersects')
MS_OR = geopandas.sjoin(F, OR, how='inner', op='intersects')
frames = [MS_WA, MS_ID, MS_OR]
MS_inside = pd.concat(frames)
MS_inside = MS_inside.drop(['gridcode_1','index_right'],axis=1)

MS_inside.shape

CPU times: user 5.81 s, sys: 420 ms, total: 6.23 s
Wall time: 6.25 s


In [286]:
## Spatial joins to get structures within refugia
## 4176 rows, 21 columns

MS_WA_ref = geopandas.sjoin(R, WA, how ='inner',op='intersects')
MS_ID_ref = geopandas.sjoin(R, ID, how='inner', op='intersects')
MS_OR_ref = geopandas.sjoin(R, OR, how='inner', op='intersects')
frames = [MS_WA_ref, MS_ID_ref, MS_OR_ref]
MS_refugia = pd.concat(frames)

MS_refugia.shape

In [296]:
## Check the burn probabilities for structures within 
## perimters versus within refugia (pretty similar)

MS_inside['bp_2016083'].describe()

count    18760.000000
mean         0.009381
std          0.010444
min          0.000000
25%          0.001797
50%          0.005891
75%          0.013678
max          0.063497
Name: bp_2016083, dtype: float64

In [297]:
MS_refugia['bp_2016083'].describe()

count    4176.000000
mean        0.009402
std         0.009748
min         0.000000
25%         0.001298
50%         0.006689
75%         0.014077
max         0.060905
Name: bp_2016083, dtype: float64

In [309]:
# %%time
# MS_inside.to_file('/data/yoder/RefugiaProject/Microsoft_inside.shp')
# MS_refugia.to_file('/data/yoder/RefugiaProject/Microsoft_refugia.shp')

In [ ]:
## Load Zillow Data
## This will take a while 

In [319]:
# Create Dictionary for State Abbreviations and Zillow numbers
Zillowpath = '/data/yoder/ZillowAccessor_2001_2014/'
os.chdir(Zillowpath)

States = {}
for p in glob.glob(Zillowpath+'*'):  
    try:
        df = pd.read_csv(p+'/ZAsmt\Main.txt', delimiter="|", header=None, nrows=100)
        States[p] = df.iloc[0,3]
    except:
        pass
        
st = {value:key for key, value in States.items()}
# dict(sorted(st.items()))
## Select only Western States and create dictionary for 
West = ['WA', 'OR', 'ID']
# ShortWest =['WA', 'OR', 'CA', 'ID','NV']
WestStatesList = [st[x] for x in st if x in West]
WestStatesDict = {x:st[x] for x in West}
WestStatesDict

{'WA': '/data/yoder/ZillowAccessor_2001_2014/53',
 'OR': '/data/yoder/ZillowAccessor_2001_2014/41',
 'ID': '/data/yoder/ZillowAccessor_2001_2014/16'}

In [320]:
# ####  Merge to create one dataframe for each state:

## Change last digits to desired state
subpath = '/data/yoder/ZillowAccessor_2001_2014/16'
os.chdir(subpath)
os.listdir()

Main=pd.read_csv("ZAsmt\\Main.txt", sep="|", header=None) #, encoding='latin1')
print("Loaded Main")
Building=pd.read_csv("ZAsmt\\Building.txt", sep="|", header=None)
print("Loaded Building")
BuildingAreas=pd.read_csv("ZAsmt\\BuildingAreas.txt", sep="|", header=None)
print("Loaded Building Areas")
Garage=pd.read_csv("ZAsmt\\Garage.txt", sep= "|", header=None)
print("Loaded Garage")
# LotSiteAppeal=pd.read_csv("ZAsmt\\LotSiteAppeal.txt", sep="|", header=None)
# print("Loaded LotSiteAppeal")
Value=pd.read_csv("ZAsmt\\Value.txt", sep="|", header=None)
print("Loaded Value")

## Label columns using excel table with list of columnn names
def NameCols(dataframe, dictname):
    xls=pd.ExcelFile(f'/data/yoder/ZillowAccessor_2001_2014/ZAsmt_dicts/{dictname}.xlsx')
    df=xls.parse(xls.sheet_names[0])
    df.to_dict()
    dataframe.columns = dataframe.columns.to_series().map(df['FieldName'])
    
NameCols(Main, 'utMain')   
NameCols(Building, 'utBuilding')
NameCols(BuildingAreas, 'utBuildingAreas')
NameCols(Garage, 'utGarage')
NameCols(Value, 'utValue')
# NameCols(LotSiteAppeal,'utLotSiteAppeal')


All=pd.merge(Main, Building, on='RowID')
All=pd.merge(Main, BuildingAreas, on='RowID')
All=pd.merge(Main, Garage, on='RowID')
All=pd.merge(Main, Value, on='RowID')


ID_proj = geopandas.GeoDataFrame(All, geometry=geopandas.points_from_xy(All.PropertyAddressLongitude, All.PropertyAddressLatitude))
ID_proj.crs="epsg:4326"
ID_proj=ID_proj.to_crs("EPSG:2163")

ID_inside = geopandas.sjoin(ID_proj, F, how ='inner', op='intersects')

Loaded Main
Loaded Building
Loaded Building Areas
Loaded Garage
Loaded Value


In [347]:
ID_inside_rows = ID_inside[['RowID', 'geometry']].copy()
ID_inside_rows.to_file("/data/yoder/RefugiaProject/ID_rows_inside.shp")

In [ ]:
# ####  Merge to create one dataframe for each state:

## Change last digits to desired state
subpath = '/data/yoder/ZillowAccessor_2001_2014/53'
os.chdir(subpath)
os.listdir()

Main=pd.read_csv("ZAsmt\\Main.txt", sep="|", header=None, encoding='latin1')
print("Loaded Main")
Building=pd.read_csv("ZAsmt\\Building.txt", sep="|", header=None)
print("Loaded Building")
BuildingAreas=pd.read_csv("ZAsmt\\BuildingAreas.txt", sep="|", header=None)
print("Loaded Building Areas")
Garage=pd.read_csv("ZAsmt\\Garage.txt", sep= "|", header=None)
print("Loaded Garage")
# LotSiteAppeal=pd.read_csv("ZAsmt\\LotSiteAppeal.txt", sep="|", header=None)
# print("Loaded LotSiteAppeal")
Value=pd.read_csv("ZAsmt\\Value.txt", sep="|", header=None)
print("Loaded Value")

## Label columns using excel table with list of columnn names
def NameCols(dataframe, dictname):
    xls=pd.ExcelFile(f'/data/yoder/ZillowAccessor_2001_2014/ZAsmt_dicts/{dictname}.xlsx')
    df=xls.parse(xls.sheet_names[0])
    df.to_dict()
    dataframe.columns = dataframe.columns.to_series().map(df['FieldName'])
    
NameCols(Main, 'utMain')   
NameCols(Building, 'utBuilding')
NameCols(BuildingAreas, 'utBuildingAreas')
NameCols(Garage, 'utGarage')
NameCols(Value, 'utValue')
# NameCols(LotSiteAppeal,'utLotSiteAppeal')
print("Named All")

All=pd.merge(Main, Building, on='RowID')
All=pd.merge(Main, BuildingAreas, on='RowID')
All=pd.merge(Main, Garage, on='RowID')
All=pd.merge(Main, Value, on='RowID')
print("Merged All")


ID_proj = geopandas.GeoDataFrame(All, geometry=geopandas.points_from_xy(All.PropertyAddressLongitude, All.PropertyAddressLatitude))
ID_proj.crs="epsg:4326"
print("Set CRS")
ID_proj=ID_proj.to_crs("EPSG:2163")
print("Changed CRS")

WA_inside = geopandas.sjoin(ID_proj, F, how ='inner', op='intersects')

/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,9,10,11,13,15,17,19,20,22,25,29,30,34,37,43,44,45,46,47,49,50,52,53,54,55,56,57,59,60,61,63,64,65,67,68,69,73,74,77,78,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loaded Main


/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,7,9,10,12,13,17,29,30,31,32,33,34,35,38,39,40,41,42,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loaded Building
Loaded Building Areas
Loaded Garage
Loaded Value
Named All
Merged All
Set CRS
Changed CRS


In [351]:
WA_inside_rows = WA_inside[['RowID', 'geometry']].copy()
WA_inside_rows.to_file("/data/yoder/RefugiaProject/WA_rows_inside.shp")

In [ ]:
%%time
# ####  Merge to create one dataframe for each state:

## Change last digits to desired state
subpath = '/data/yoder/ZillowAccessor_2001_2014/41'
os.chdir(subpath)
os.listdir()

Main=pd.read_csv("ZAsmt\\Main.txt", sep="|", header=None, encoding='latin1')
print("Loaded Main")
Building=pd.read_csv("ZAsmt\\Building.txt", sep="|", header=None)
print("Loaded Building")
BuildingAreas=pd.read_csv("ZAsmt\\BuildingAreas.txt", sep="|", header=None)
print("Loaded Building Areas")
Garage=pd.read_csv("ZAsmt\\Garage.txt", sep= "|", header=None)
print("Loaded Garage")
# LotSiteAppeal=pd.read_csv("ZAsmt\\LotSiteAppeal.txt", sep="|", header=None)
# print("Loaded LotSiteAppeal")
Value=pd.read_csv("ZAsmt\\Value.txt", sep="|", header=None)
print("Loaded Value")

## Label columns using excel table with list of columnn names
def NameCols(dataframe, dictname):
    xls=pd.ExcelFile(f'/data/yoder/ZillowAccessor_2001_2014/ZAsmt_dicts/{dictname}.xlsx')
    df=xls.parse(xls.sheet_names[0])
    df.to_dict()
    dataframe.columns = dataframe.columns.to_series().map(df['FieldName'])
    
NameCols(Main, 'utMain')   
NameCols(Building, 'utBuilding')
NameCols(BuildingAreas, 'utBuildingAreas')
NameCols(Garage, 'utGarage')
NameCols(Value, 'utValue')
# NameCols(LotSiteAppeal,'utLotSiteAppeal')
print("Named All")

All=pd.merge(Main, Building, on='RowID')
All=pd.merge(Main, BuildingAreas, on='RowID')
All=pd.merge(Main, Garage, on='RowID')
All=pd.merge(Main, Value, on='RowID')
print("Merged All")


ID_proj = geopandas.GeoDataFrame(All, geometry=geopandas.points_from_xy(All.PropertyAddressLongitude, All.PropertyAddressLatitude))
ID_proj.crs="epsg:4326"
print("Set CRS")
ID_proj=ID_proj.to_crs("EPSG:2163")
print("Changed CRS")

OR_inside = geopandas.sjoin(ID_proj, F, how ='inner', op='intersects')

In [ ]:
%%time
ID_proj = ID_proj.to_crs("EPSG:2163")

In [ ]:
%%time
OR_inside = geopandas.sjoin(ID_proj, F, how ='inner', op='intersects')

In [ ]:
OR_inside_rows = OR_inside[['RowID', 'geometry']].copy()
OR_inside_rows.to_file("/data/yoder/RefugiaProject/OR_rows_inside.shp")

In [369]:
%%time
# os.chdir('/data/yoder/RefugiaProject/RefugiaZillow')

# ID_wFires.to_file('ID_withinRefugiaFires.shp')
# OR_wFires.to_file('OR_withinRefugiaFires.shp')
# WA_wFires.to_file('WA_withinRefugiaFires.shp')
frames = [ID_inside, OR_inside, WA_inside]
Zillow_inside = pd.concat(frames)
# Z_inside.to_file('Zillow_inside_RefugiaFires.shp')

CPU times: user 158 ms, sys: 0 ns, total: 158 ms
Wall time: 169 ms


In [375]:
Z_inside = Zillow_inside.copy()

In [376]:
Z_inside = Z_inside.sort_values(by=['AssessmentYear'], ascending=False)

In [377]:
## Keep only structures with assessed values prior to the fire and delete all but most recent recod
Z_inside = Z_inside.loc[(Z_inside['AssessmentYear'] <= Z_inside['Year'])]
Z_inside = Z_inside.drop_duplicates(subset=['geometry'], keep='first')
Z_inside = Z_inside.drop(['index_right'],axis=1)
Z_refugia = geopandas.sjoin(R, Z_inside, how='inner', op='intersects')

In [381]:
Z_inside_rows = Z_inside[['RowID','geometry']].copy()

In [382]:
Z_refugia_rows = Z_refugia[['RowID','geometry']].copy()

In [384]:
Z_refugia = Z_refugia.drop(['geometry_x','geometry_y'],axis=1)

In [385]:
# Z_inside_rows.to_file('/data/yoder/RefugiaProject/RefugiaZillow/Z_inside.shp')
# Z_refugia_rows.to_file('/data/yoder/RefugiaProject/RefugiaZillow/Z_refugia.shp')

In [387]:
Z_inside.shape

(1869, 158)

In [388]:
Z_refugia.shape

(266, 163)

In [407]:
Z_inside[['TotalAssessedValue','LandAssessedValue','LandMarketValue','ImprovementMarketValue']].sum()

TotalAssessedValue        385927227.0
LandAssessedValue          84171907.0
LandMarketValue           130244816.0
ImprovementMarketValue    197296448.0
dtype: float64

In [408]:
Z_refugia[['TotalAssessedValue','LandAssessedValue','LandMarketValue','ImprovementMarketValue']].sum()

TotalAssessedValue        50442433.0
LandAssessedValue         12882532.0
LandMarketValue           19211117.0
ImprovementMarketValue    32597613.0
dtype: float64

In [ ]:
# basemap = Fires.plot(edgecolor='red',color='pink')
# Refugia.plot(ax=basemap, color='lightblue')
# MS_inside.plot(ax=basemap, color = 'blue')
# Alex_in.plot(ax=basemap, color = 'green')
# Z_inside.plot(ax=basemap,color = 'cyan')

In [413]:
A_counts = Alex_in.groupby(['Fire_ID']).sum()[['BH','NH','RH','UH','UK']]

In [414]:
A_counts

,BH,NH,RH,UH,UK
Fire_ID,,,,,
ID4206511258020000916,0.0,0.0,0.0,36.0,0.0
ID4208311299720060703,12.0,0.0,0.0,21.0,0.0
ID4215011379020000716,0.0,0.0,0.0,0.0,4.0
ID4217911557220000626,0.0,0.0,0.0,4.0,0.0
ID4224811280920110809,0.0,0.0,0.0,80.0,0.0
...,...,...,...,...,...
WA4869311918120080628,0.0,0.0,0.0,16.0,0.0
WA4878711937120020724,0.0,0.0,0.0,189.0,252.0
WA4884211935820000722,0.0,0.0,0.0,9315.0,4140.0


In [433]:
MS_inside.head(1)

,ValidMTBS,MTBS_Name,Fire_NO,Fire_ID,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,geometry_x,fire_id,ID,GRIDCODE,F_AREA,geometry_y,geometry,FID_1,MSID,ORIG_FID,bp_2016083,fil1_20160,fil2_20160,fil3_20160,fil4_20160,fil5_20160,fil6_20160,OBJECTID,Shape_Leng,Shape_Area
1722,WA4564012112620100712,DALLESPORT,68,WA4564012112620100712,2010193,45,28,10,4.797,0.2826,5.89118,2.94559,7.72983,0.356473,87.9925,1.42589,1.9137,0.581613,LT50450282009246PAC01,LE70450282011244EDC00,LT50450282009246PAC01,LT50450282010201PAC01,0,0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.002295,0.003295,0.0,0.000687,DALLESPORT,2010,7,12,Wildfire,1177.0,"POLYGON ((-1619003.467 286048.446, -1618964.01...",WA4564012112620100712,11099,68,2700.0,MULTIPOLYGON Z (((-1618659.112 283926.608 0.00...,"POLYGON ((-1619003.467 286048.446, -1618964.01...",960149.0,WA_960149,960149,0.002696,0.0,0.296296,0.481481,0.222222,0.0,0.0,NaN,NaN,NaN


In [435]:
Alex_in.head(1)

,Name,PopupInfo,YEAR,TYPE_H,Area,Perimeter,Acres_a,Fire_ID,Fire_Name_a,StartMonth_a,StartDay_a,Confidence,Comment,FireType,Distance,STATEABREV,NA_L2CODE,NA_L2NAME,NA_L1CODE,NA_L1NAME,NA_L2KEY,NA_L1KEY,geometry_a,ValidMTBS,MTBS_Name,Fire_NO,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name_f,Year,StartMonth_f,StartDay_f,Fire_Type,Acres_f,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry_f,geometry,Dist_perim
0,UK_2000_037,1998;2002;2002;3,2000,UK,6.436686e+06,18276.20788,1590.539934,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.0,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275523.152 -63536.007 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.4854,0.1701,2.62281,0.0,0.527338,12.101,87.3161,0.013877,0.041632,0.0,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.0,1,0.0,0.0,0.0,0.0,6.0,0.044076,0.054027,0.0,0.005772,MP 77 I-84,2000,7,17,Wildfire,1591.0,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.0,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275523.152 -63536.007 0.000),0.0


In [454]:
# %%time
# S1 = geopandas.read_file('/data/yoder/RefugiaProject/RandomPoints/10m_joined.shp')
# S2 = geopandas.read_file('/data/yoder/RefugiaProject/RandomPoints/50m_joined.shp')

CPU times: user 8min 23s, sys: 30.5 s, total: 8min 54s
Wall time: 8min 57s


In [458]:
# S1 = S1.drop(['TARGET_FID','CID','GRIDCODE','gridcode_1','geometry'], axis=1)
# S2 = S2.drop(['TARGET_FID','CID','GRIDCODE','gridcode_1','geometry'], axis=1)

In [479]:
N_alex = 4414
N_MS = 18760
N_Zillow = 1869

In [1281]:
Alex_buffer = Alex.copy()

In [1284]:
Alex_buffer['geometry'] = Alex_buffer.geometry.buffer(16.9257)

In [1287]:
Alex_buffer.area.sum()

6495663.349951982

In [1288]:
F.area.sum()

90672935119.15356

In [1289]:
R.area.sum()

7613872315.119953

In [1300]:
%%time
ABuff_joinRef = geopandas.sjoin(Ref, Alex_buffer, how='right',op='intersects' )

CPU times: user 37.6 s, sys: 777 ms, total: 38.4 s
Wall time: 38.5 s


In [1299]:
Alex_buffer['uniqueVal'] = Alex_buffer.index

In [1305]:
ABuff_joinRef['index_left'].count()

2433

In [1306]:
Alex.shape

(7229, 23)

In [491]:
Rpoints = S1.append(S2)

In [507]:
Rpoints = Rpoints.drop(['ID','fire_id','F_AREA'],axis=1)

In [602]:
# Rpoints

In [ ]:
## Sampling takes 29.3 seconds for 100 samples n = N_alex (S1)
## Sampling takes 36 seconds for 100 samples n = N_alex (S2)
## Sampling takes 1 min 25 sec for 100 samples with Rpoints

## MUCH FASTER WITH PYTHON RANDOM
pop = list(justpoints)

In [541]:
justpoints = Rpoints['Join_Count'].copy()

In [625]:
# %%time
## SLOW

# List_samples =  []
# for i in range(100):
#     List_samples.append(justpoints.sample(n=N_MS).sum())

CPU times: user 1min 24s, sys: 1.66 s, total: 1min 25s
Wall time: 1min 26s


In [1276]:
%%time
## FAST !
List_samples = []
for i in range(10000):
    subsamp = []
    for i in range(N_alex):
        subsamp.append(random.choice(pop))
    List_samples.append(np.asarray(subsamp).sum())

CPU times: user 46.8 s, sys: 848 ms, total: 47.7 s
Wall time: 48 s


In [1277]:
l1 = np.asarray(List_samples)

In [1278]:
l1.max()

530

In [1279]:
np.percentile(l1, 99)

502.0

In [ ]:

ls.percentils)

In [1252]:
Z_inside.shape

(1869, 158)

In [1262]:
Z_refugia.shape

(266, 163)

In [1591]:
PNW = F.copy()

In [1592]:
PNW[['area(ha)','Unburned(h']].sum()*1000

area(ha)     90985746.00000
Unburned(h    8389295.50000
dtype: float64

In [1593]:
PNW.head(1)

,ValidMTBS,MTBS_Name,Fire_NO,Fire_ID,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry
0,OR4239211789420120708,LONG DRAW,85,OR4239211789420120708,2012190,42,31,11,2255.49000,92.31930,4.09310,21.10830,0.90811,0.04034,98.04380,0.85886,0.13846,0.01045,LT50420312011247PAC01,LC80420312013252LGN00,LT50420312011215PAC01,LE70420312012226EDC00,0,0,0.00000,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.01024,0.01518,0.00000,0.00161,LONG DRAW,2012,7,8,Wildfire,557620.00000,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",OR4239211789420120708,215452,85,85,1800.00000,MULTIPOLYGON Z (((-1442956.036 -156929.659 0.0...,"POLYGON ((-1437231.106 -106411.878, -1436471.2..."


In [1594]:
PNW['Unburned%']

0       4.09310
1      15.58560
2       3.18784
3       7.59000
4       4.88811
         ...   
2048    2.44030
2049    3.08841
2050   12.37370
2051    2.96751
2052   12.58080
Name: Unburned%, Length: 2053, dtype: float64

In [1595]:
PNW = PNW.rename({'Unburned%': 'PctRefugia', 'Unburned(h':'UnburnedHA', 'Masked%': 'PctMasked', 
                  'Tree%': 'PctTree',  'Shrub%':'PctShrub',  'Herb%':'PctHerb',
                  'Water/Nodata%':'PctWaterND',  'Barren/Spa':'PctBarren',
                 'Ag/Buildup': 'PctAgbuild', 'Water/Noda':'PctWaterND', 'area(ha)':'AreaHA'}, axis=1)  # new method

In [1596]:
PNW['Year'] =  PNW['Fire_DOY'].astype(str).str[0:4]
PNW['DOY'] = PNW['Fire_DOY'].astype(str).str[4:7]
PNW['DOY'] = PNW['DOY'].astype(int)

In [1597]:
PNW.head(1)

,ValidMTBS,MTBS_Name,Fire_NO,Fire_ID,Fire_DOY,Path,Row,Zone,AreaHA,UnburnedHA,PctRefugia,PctMasked,PctTree,PctShrub,PctHerb,PctBarren,PctAgbuild,PctWaterND,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry,DOY
0,OR4239211789420120708,LONG DRAW,85,OR4239211789420120708,2012190,42,31,11,2255.49000,92.31930,4.09310,21.10830,0.90811,0.04034,98.04380,0.85886,0.13846,0.01045,LT50420312011247PAC01,LC80420312013252LGN00,LT50420312011215PAC01,LE70420312012226EDC00,0,0,0.00000,1,0.00000,0.00000,0.00000,0.00000,0.00000,0.01024,0.01518,0.00000,0.00161,LONG DRAW,2012,7,8,Wildfire,557620.00000,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",OR4239211789420120708,215452,85,85,1800.00000,MULTIPOLYGON Z (((-1442956.036 -156929.659 0.0...,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",190


In [1598]:
Z_counts = Z_inside[['State','Fire_ID','ImprovementMarketValue']].copy()
Z_counts['Z_count'] = 1
Z_counts = Z_counts.groupby(['Fire_ID']).sum()

In [1599]:
Z_counts

,ImprovementMarketValue,Z_count
Fire_ID,,
ID4225911218820121020,0.00000,3
ID4238111461920130629,0.00000,1
ID4261811618520120709,21817.00000,2
ID4274711279320110821,7000.00000,4
ID4275711507020100821,0.00000,3
...,...,...
WA4821112010220140714,71581400.00000,641
WA4825011939020110829,354600.00000,3
WA4833111962320130715,418500.00000,4


In [1600]:
MS_counts = MS_inside[['bp_2016083','Fire_ID']].copy()
MS_counts['MS_count'] = 1
MS_counts = MS_counts.groupby(['Fire_ID']).sum()

In [1601]:
MS_counts

,bp_2016083,MS_count
Fire_ID,,
ID4206011524119940718,0.01827,1
ID4206511258020000916,0.27446,14
ID4207311108819991009,0.00305,1
ID4207911387620100826,0.04623,2
ID4208311299720060703,0.04413,2
...,...,...
WA4888211767920030810,0.00080,1
WA4896311932420030630,0.06679,36
WA4896412045520020817,0.00839,1


In [1602]:
Alex_count = Alex_in[['Fire_ID','BH','NH','RH','UH','UK']].copy()

In [1603]:
Alex_count = Alex_count.drop_duplicates('Fire_ID')

In [1604]:
Alex_count

,Fire_ID,BH,NH,RH,UH,UK
0,ID4330711588920000717,0.00000,0.00000,0.00000,0.00000,6.00000
6,ID4320511555120000807,0.00000,0.00000,0.00000,22.00000,28.00000
56,ID4355111606420000924,0.00000,0.00000,0.00000,4.00000,9.00000
69,ID4217911557220000626,0.00000,0.00000,0.00000,2.00000,0.00000
71,ID4355611308220000917,0.00000,0.00000,0.00000,3.00000,0.00000
...,...,...,...,...,...,...
6636,WA4729912010620130727,0.00000,0.00000,0.00000,10.00000,0.00000
6646,ID4334011550020130809,21.00000,0.00000,0.00000,57.00000,0.00000
6724,OR4481412123520130720,0.00000,0.00000,0.00000,25.00000,0.00000
6749,WA4606511698720130608,0.00000,0.00000,0.00000,25.00000,0.00000


In [1605]:
PNW = pd.merge(PNW, MS_counts, how='left',on='Fire_ID')

In [1606]:
PNW = pd.merge(PNW, Z_counts, how='left',on='Fire_ID')

In [1607]:
PNW = PNW.drop(['BH', 'NH', 'RH', 'UH', 'UK'],axis =1)

In [1608]:
PNW = pd.merge(PNW, Alex_count, how ='left',on='Fire_ID')

In [1609]:
PNW[['BH', 'NH', 'RH', 'UH', 'UK', 'MS_count','ImprovementMarketValue','Z_count']] = PNW[['BH', 'NH', 'RH', 'UH', 'UK', 'MS_count','ImprovementMarketValue','Z_count']].fillna(0)

In [1610]:
Alex_in['UH'].describe()

count   4865.00000
mean     126.14717
std      170.05156
min        0.00000
25%       22.00000
50%       57.00000
75%      109.00000
max      555.00000
Name: UH, dtype: float64

In [1611]:
Alex_in

,Name,PopupInfo,YEAR,TYPE_H,Area,Perimeter,Acres_a,Fire_ID,Fire_Name_a,StartMonth_a,StartDay_a,Confidence,Comment,FireType,Distance,STATEABREV,NA_L2CODE,NA_L2NAME,NA_L1CODE,NA_L1NAME,NA_L2KEY,NA_L1KEY,geometry_a,ValidMTBS,MTBS_Name,Fire_NO,Fire_DOY,Path,Row,Zone,area(ha),Unburned(h,Unburned%,Masked%,Tree%,Shrub%,Herb%,Barren/Spa,Ag/Buildup,Water/Noda,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,BH,NH,RH,UH,UK,MEAN,MAX,MIN,STD,Fire_Name_f,Year,StartMonth_f,StartDay_f,Fire_Type,Acres_f,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry_f,geometry,Dist_perim
0,UK_2000_037,1998;2002;2002;3,2000,UK,6436686.26501,18276.20788,1590.53993,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.00000,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275523.152 -63536.007 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.48540,0.17010,2.62281,0.00000,0.52734,12.10100,87.31610,0.01388,0.04163,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.00000,1,0.00000,0.00000,0.00000,0.00000,6.00000,0.04408,0.05403,0.00000,0.00577,MP 77 I-84,2000,7,17,Wildfire,1591.00000,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.00000,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275523.152 -63536.007 0.000),0.00000
1,UK_2000_037,1998;2002;2002;3,2000,UK,6436686.26501,18276.20788,1590.53993,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.00000,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275291.851 -63337.757 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.48540,0.17010,2.62281,0.00000,0.52734,12.10100,87.31610,0.01388,0.04163,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.00000,1,0.00000,0.00000,0.00000,0.00000,6.00000,0.04408,0.05403,0.00000,0.00577,MP 77 I-84,2000,7,17,Wildfire,1591.00000,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.00000,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275291.851 -63337.757 0.000),0.00000
2,UK_2000_037,1998;2002;2002;3,2000,UK,6436686.26501,18276.20788,1590.53993,ID4330711588920000717,MP 77 I-84,7,17,High,None,WF,0.00000,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1275320.547 -64011.052 0.000),ID4330711588920000717,MP 77 I-84,44,2000199,41,30,11,6.48540,0.17010,2.62281,0.00000,0.52734,12.10100,87.31610,0.01388,0.04163,0.00000,LT50410302000194AAA03,LE70410302001204EDC00,LT50410301999207XXX04,LE70410302000218EDC00,0,0,0.00000,1,0.00000,0.00000,0.00000,0.00000,6.00000,0.04408,0.05403,0.00000,0.00577,MP 77 I-84,2000,7,17,Wildfire,1591.00000,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",ID4330711588920000717,119525,44,44,1800.00000,MULTIPOLYGON Z (((-1274319.573 -65741.537 0.00...,"POLYGON ((-1274021.740 -65710.234, -1274078.82...",POINT Z (-1275320.547 -64011.052 0.000),0.00000
6,UH_2000_039,1998;2003;2003;3,2000,UH,127777857.05300,71253.13990,31574.59848,ID4320511555120000807,OREGON TRAIL,8,7,High,None,WF,0.00000,ID,10.1,COLD DESERTS,10,NORTH AMERICAN DESERTS,10.1 COLD DESERTS,10 NORTH AMERICAN DESERTS,POINT Z (-1254228.870 -80538.673 0.000),ID4320511555120000807,OREGON TRAIL,41,2000220,41,30,11,127.83300,4.51350,3.53077,3.85533,0.42172,63.49470,34.70790,0.39708,0.97510,0.00352,LE70410302000218EDC00,LT50410302001228LGS01,LE70410301999231EDC00,LT50410302000226XXX03,0,0,0.00000,1,0.00000,0.00000,0.00000,22.00000,28.00000,0.03337,0.04785,0.00000,0.00622,OREGON TRAIL,2000,8,7,Wildfire,31575.00000,"POLYGON ((-1259155.779 -72534.356, -1259136.65...",ID4320511555120000807,120490,41,41,3600.00000,MULTIPOLYGON Z (((-1246279.909 -89925.884 0.00...,"POLYGON ((-1259155

In [1612]:
PNW

,ValidMTBS,MTBS_Name,Fire_NO,Fire_ID,Fire_DOY,Path,Row,Zone,AreaHA,UnburnedHA,PctRefugia,PctMasked,PctTree,PctShrub,PctHerb,PctBarren,PctAgbuild,PctWaterND,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,MEAN,MAX,MIN,STD,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry,DOY,bp_2016083,MS_count,ImprovementMarketValue,Z_count,BH,NH,RH,UH,UK
0,OR4239211789420120708,LONG DRAW,85,OR4239211789420120708,2012190,42,31,11,2255.49000,92.31930,4.09310,21.10830,0.90811,0.04034,98.04380,0.85886,0.13846,0.01045,LT50420312011247PAC01,LC80420312013252LGN00,LT50420312011215PAC01,LE70420312012226EDC00,0,0,0.00000,1,0.01024,0.01518,0.00000,0.00161,LONG DRAW,2012,7,8,Wildfire,557620.00000,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",OR4239211789420120708,215452,85,85,1800.00000,MULTIPOLYGON Z (((-1442956.036 -156929.659 0.0...,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",190,0.14976,26.00000,37970.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,ID4202911394119961001,SHOEPASTUR,1,ID4202911394119961001,1996275,40,31,11,8.06130,1.25640,15.58560,0.00000,14.67010,80.27240,5.05750,0.00000,0.00000,0.00000,LT50400311996224XXX02,LT50400311997194XXX01,LT50400311995221XXX01,LT50400311996288XXX03,1,0,0.00000,1,0.03203,0.03834,0.02296,0.00336,SHOEPASTUR,1996,10,1,Prescribed Fire,1982.00000,"POLYGON ((-1144578.205 -230478.566, -1144515.7...",ID4202911394119961001,101993,1,1,22500.00000,MULTIPOLYGON Z (((-1144988.481 -234420.465 0.0...,"POLYGON ((-1144578.205 -230478.566, -1144515.7...",275,nan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,ID4204811467420120710,RABBIT SPRINGS,1,ID4204811467420120710,2012192,40,31,11,6.04170,0.19260,3.18784,10.51690,0.00000,0.49158,93.71370,1.43006,4.36467,0.00000,LT50400312011249PAC01,LC80400312013222LGN00,LT50400312011217PAC01,LE70400312012212EDC00,0,0,0.00000,1,0.02005,0.02785,0.01438,0.00282,RABBIT SPRINGS,2012,7,10,Wildfire,1484.00000,"POLYGON ((-1204197.255 -219931.978, -1204170.2...",ID4204811467420120710,278750,1,1,3600.00000,MULTIPOLYGON Z (((-1204609.802 -224565.500 0.0...,"POLYGON ((-1204197.255 -219931.978, -1204170.2...",192,nan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,ID4205411588719890727,BRUSH CK,1,ID4205411588719890727,1989208,41,31,11,10.62450,0.80640,7.59000,0.00000,12.82510,87.07330,0.05083,0.03388,0.00000,0.01694,LT50410311988241XXX02,LT50410311991153XXX03,LT50410311988241XXX02,LT50410311989227XXX02,0,0,0.00000,1,0.00535,0.00819,0.00280,0.00148,BRUSH CK,1989,7,27,Wildfire,2616.00000,"POLYGON ((-1301187.030 -198957.525, -1301143.0...",ID4205411588719890727,26784,1,1,1800.00000,MULTIPOLYGON Z (((-1303550.276 -202406.490 0.0...,"POLYGON ((-1301187.030 -198957.525, -1301143.0...",208,nan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,ID4205911398420000817,BURLEY COMPLEX (COAL BANKS COMPLEX),3,ID4205911398420000817,2000230,40,31,11,33.82290,1.65330,4.88811,0.00000,36.64620,62.22560,1.06969,0.04524,0.01330,0.00000,LT50400312000203XXX02,LT50400312001205LGS01,LT50400311999248XXX01,LT50400312000251XXX02,0,0,0.00000,1,0.03145,0.03914,0.00000,0.00456,BURLEY COMPLEX (COAL BANKS COMPLEX),2000,8,17,Wildfire,8330.00000,"POLYGON ((-1151558.458 -231161.271, -1151574.5...",ID4205911398420000817,133782,3,3,6300.00000,MULTIPOLYGON Z (((-1149460.378 -232466.139 0.0...,"POLYGON ((-1151558.458 -231161.271, -1151574.5...",230,nan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2048,WA4896311932420030630,UNNAMED,89,WA4896311932420030630,2003181,45,26,11,6.89670,0.16830,2.44030,0.00000,9.01736,0.57419,89.39060,1.01788,0.00000,0.00000,LE70450262002235EDC00,LT50450262004233PAC01,LE70450262002235EDC00,LT50450262003230PAC02

In [1613]:
PNW = PNW[PNW['AreaHA']>.404]

In [1614]:
PNW.shape

(2050, 54)

In [1615]:
PNW['ZperHA'] = PNW['Z_count']/PNW['AreaHA']
PNW['MS_perHA'] = PNW['MS_count']/PNW['AreaHA']
PNW['A_BHperHA'] = PNW['BH']/PNW['AreaHA']
PNW['A_UHperHA'] = PNW['UH']/PNW['AreaHA']
PNW['ImprovPerHA'] = PNW['ImprovementMarketValue']/PNW['AreaHA']

/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [1616]:
PNW['Year'] = PNW['Year'].astype(int)

/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1617]:
PNW.head(1)

,ValidMTBS,MTBS_Name,Fire_NO,Fire_ID,Fire_DOY,Path,Row,Zone,AreaHA,UnburnedHA,PctRefugia,PctMasked,PctTree,PctShrub,PctHerb,PctBarren,PctAgbuild,PctWaterND,TM_pre_ID,TM_post_id,TM_pre_ID_,TM_post__1,QC_Flag,roi_id,Greenup%,2yrs_avail,MEAN,MAX,MIN,STD,Fire_Name,Year,StartMonth,StartDay,Fire_Type,Acres,geometry_x,fire_id,ID,GRIDCODE,gridcode_1,F_AREA,geometry_y,geometry,DOY,bp_2016083,MS_count,ImprovementMarketValue,Z_count,BH,NH,RH,UH,UK,ZperHA,MS_perHA,A_BHperHA,A_UHperHA,ImprovPerHA
0,OR4239211789420120708,LONG DRAW,85,OR4239211789420120708,2012190,42,31,11,2255.49000,92.31930,4.09310,21.10830,0.90811,0.04034,98.04380,0.85886,0.13846,0.01045,LT50420312011247PAC01,LC80420312013252LGN00,LT50420312011215PAC01,LE70420312012226EDC00,0,0,0.00000,1,0.01024,0.01518,0.00000,0.00161,LONG DRAW,2012,7,8,Wildfire,557620.00000,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",OR4239211789420120708,215452,85,85,1800.00000,MULTIPOLYGON Z (((-1442956.036 -156929.659 0.0...,"POLYGON ((-1437231.106 -106411.878, -1436471.2...",190,0.14976,26.00000,37970.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00089,0.01153,0.00000,0.00000,16.83448


In [1618]:
NumRefugia = Refugia.copy()
NumRefugia['NumRefugia'] = 1
NumRefugia = NumRefugia.drop(['ID','GRIDCODE','gridcode_1','geometry'],axis=1)
NumRefugia = NumRefugia.groupby(['fire_id']).sum()
PNW = PNW.merge(NumRefugia, how='left',left_on='Fire_ID',right_on='fire_id')

In [1619]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [1620]:
F.area.sum()

90672935119.15356

In [1621]:
PN['AreaHA'].sum()


90985.4103

In [1622]:
PN['UnburnedHA'].sum()

8389.2739

In [1623]:
PNW[['Tree%','Shrub%','Herb%','Barren/Spa','Ag/Buildup','Water/Noda']].describe().loc[['count','mean','std','min','max']].T

KeyError: "None of [Index(['Tree%', 'Shrub%', 'Herb%', 'Barren/Spa', 'Ag/Buildup', 'Water/Noda'], dtype='object')] are in the [columns]"

In [1624]:
PN = PNW[['Fire_ID','AreaHA', 'UnburnedHA', 'PctRefugia','PctMasked','PctTree','PctShrub','PctHerb',
          'PctWaterND','MEAN','Year','DOY','MS_count','ImprovementMarketValue','Z_count','BH','NH','RH','UH','UK',
         'ZperHA', 'MS_perHA','A_BHperHA','A_UHperHA','ImprovPerHA','NumRefugia']]

In [1625]:
PN.shape

(2050, 26)

In [1626]:
PN['RefugiaArea'] = PN['AreaHA']*(PN['PctRefugia']/100)

/opt/apps/anaconda3/20.02.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [1628]:
PN

,Fire_ID,AreaHA,UnburnedHA,PctRefugia,PctMasked,PctTree,PctShrub,PctHerb,PctWaterND,MEAN,Year,DOY,MS_count,ImprovementMarketValue,Z_count,BH,NH,RH,UH,UK,ZperHA,MS_perHA,A_BHperHA,A_UHperHA,ImprovPerHA,NumRefugia,RefugiaArea
0,OR4239211789420120708,2255.49000,92.31930,4.09310,21.10830,0.90811,0.04034,98.04380,0.01045,0.01024,2012,190,26.00000,37970.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00089,0.01153,0.00000,0.00000,16.83448,4816,92.31946
1,ID4202911394119961001,8.06130,1.25640,15.58560,0.00000,14.67010,80.27240,5.05750,0.00000,0.03203,1996,275,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,99,1.25640
2,ID4204811467420120710,6.04170,0.19260,3.18784,10.51690,0.00000,0.49158,93.71370,0.00000,0.02005,2012,192,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,28,0.19260
3,ID4205411588719890727,10.62450,0.80640,7.59000,0.00000,12.82510,87.07330,0.05083,0.01694,0.00535,1989,208,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,74,0.80640
4,ID4205911398420000817,33.82290,1.65330,4.88811,0.00000,36.64620,62.22560,1.06969,0.00000,0.03145,2000,230,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,224,1.65330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2045,WA4896311932420030630,6.89670,0.16830,2.44030,0.00000,9.01736,0.57419,89.39060,0.00000,0.00181,2003,181,36.00000,0.00000,0.00000,0.00000,0.00000,0.00000,5.00000,0.00000,0.00000,5.21989,0.00000,0.72498,0.00000,18,0.16830
2046,WA4896411838920030815,22.29300,0.68850,3.08841,1.06581,46.41500,0.15745,53.26600,0.00000,0.00180,2003,227,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,130,0.68850
2047,WA4896412045520020817,19.15830,2.37060,12.37370,0.17381,34.65500,0.22549,64.65920,0.00000,0.01060,2002,229,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.05220,0.00000,0.00000,0.00000,167,2.37059
2048,WA4896911967320070707,17.89380,0.53100,2.96751,0.00000,0.00000,0.00000,0.00000,0.00000,0.00064,2007,188,17.00000,0.00000,0.00000,0.00000,0.00000,0.00000,17.00000,0.00000,0.00000,0.95005,0.00000,0.95005,0.00000,53,0.53100


In [1572]:
PN['RefugiaArea'].mean()

4.092329545933995

In [1633]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [1634]:
PN.describe().loc[['count','mean','std','min','max']].T

,count,mean,std,min,max
AreaHA,2050.0000,44.3831,115.8719,4.0770,2255.4900
UnburnedHA,2050.0000,4.0923,13.1910,0.0063,241.4140
PctRefugia,2050.0000,9.6564,10.7488,0.0276,91.8238
PctMasked,2050.0000,1.4325,4.7585,0.0000,49.9001
PctTree,2050.0000,20.8832,27.6117,0.0000,99.3495
PctShrub,2050.0000,21.0287,30.2166,0.0000,100.0000
PctHerb,2050.0000,47.6094,35.0718,0.0000,100.0000
PctWaterND,2050.0000,0.1673,1.1650,0.0000,37.4181
MEAN,2050.0000,0.0143,0.0116,0.0000,0.0550
Year,2050.0000,2000.2937,8.7883,1984.0000,2014.0000


In [1347]:
Avg = PN[['PctTree','PctShrub','PctHerb','PctWaterND']].sum(axis=1)

In [1364]:
# PN['ImprovePerThousHA'] = PN['ImprovPerHA']

In [1349]:
import patsy

In [1350]:
import statsmodels.api as sm

In [1405]:
f = 'NumRefugia ~ AreaHA + AreaHA*MEAN + Year + PctTree + PctShrub + PctHerb  +  MEAN + BH + UH  + DOY + I(DOY**2) '
y , X = patsy.dmatrices(f, PN, return_type='matrix')
# resids = sm.OLS(y,X).fit().resid
# plt.scatter(y, resids)

In [1363]:
PN['NumRefugia'].sum()

621520

In [1495]:
PctRefugia_Alex = smf.ols(formula =f'PctRefugia ~ A_BHperHA + A_UHperHA  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN  + DOY + I(DOY**2)', data=PN).fit()

In [1501]:
PctRefugia_Alex.get_influence()

In [1484]:
print('Parameters: ', res.params)
print('Standard errors: ', res.bse)
print('Predicted values: ', res.predict())

Parameters:  Intercept      51.35443
A_BHperHA      -2.74988
A_UHperHA      -0.28506
AreaHA          0.00103
Year            0.02689
PctTree         0.00114
PctShrub        0.01218
PctHerb         0.01450
MEAN          -65.10216
DOY            -0.90384
I(DOY ** 2)     0.00210
dtype: float64
Standard errors:  Intercept     57.45151
A_BHperHA      3.10851
A_UHperHA      0.31929
AreaHA         0.00192
Year           0.02843
PctTree        0.01106
PctShrub       0.01122
PctHerb        0.00952
MEAN          19.58687
DOY            0.05137
I(DOY ** 2)    0.00012
dtype: float64
Predicted values:  [12.47464351 13.99466913  9.2593276  ...  8.44784751  9.19459322
  8.26584998]


In [1511]:
PctRefugia_Alex = smf.ols(formula =f'PctRefugia ~ A_BHperHA + A_UHperHA  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN  + DOY + I(DOY**2)', data=PN).fit()

In [1512]:
PctRefugia_MS = smf.ols(formula =f'PctRefugia ~ MS_perHA  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN +  DOY + I(DOY**2) ', data=PN).fit()

In [1513]:
PctRefugia_Z_count = smf.ols(formula =f'PctRefugia ~  ZperHA  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1514]:
PctRefugia_Z_value = smf.ols(formula =f'PctRefugia ~  ImprovPerHA  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1520]:
summary_col([PctRefugia_Alex, PctRefugia_MS, PctRefugia_Z_count, PctRefugia_Z_value],stars=True,regressor_order=['A_BHperHA', 'A_UHperHA', 'MS_perHA','ZperHA','ImprovPerHA', 'Intercept'], float_format='%0.4f',
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R_2':lambda x: "{:.2f}".format(x.rsquared)}).as_latex

<bound method Summary.as_latex of <class 'statsmodels.iolib.summary2.Summary'>
"""

            PctRefugia I PctRefugia II PctRefugia III PctRefugia IIII
---------------------------------------------------------------------
A_BHperHA   -2.7499                                                  
            (3.1085)                                                 
A_UHperHA   -0.2851                                                  
            (0.3193)                                                 
MS_perHA                 -0.1691**                                   
                         (0.0850)                                    
ZperHA                                 -0.3950                       
                                       (0.4997)                      
ImprovPerHA                                           -0.0000        
                                                      (0.0000)       
Intercept   51.3544      68.9312       57.1423        58.5621        
      

In [1441]:
PctRefugia_Alex = smf.ols(formula =f'PctRefugia ~ BH + UH  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1442]:
PctRefugia_MS = smf.ols(formula =f'PctRefugia ~ MS_count  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN +  DOY + I(DOY**2) ', data=PN).fit()

In [1443]:
PctRefugia_Z_count = smf.ols(formula =f'PctRefugia ~  Z_count  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1444]:
PctRefugia_Z_value = smf.ols(formula =f'PctRefugia ~ ImprovementMarketValue  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + ImprovementMarketValue  + DOY + I(DOY**2) ', data=PN).fit()

In [1456]:
summary_col([PctRefugia_Alex, PctRefugia_MS, PctRefugia_Z_count, PctRefugia_Z_value],stars=True,regressor_order=['BH','UH', 'MS_count','Z_count','ImprovementMarketValue'], float_format='%0.4f',
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)})

,PctRefugia I,PctRefugia II,PctRefugia III,PctRefugia IIII
BH,0.0574,,,
,(0.0818),,,
UH,-0.0263,,,
,(0.0175),,,
MS_count,,-0.0039,,
,,(0.0036),,
Z_count,,,-0.0106,
,,,(0.0142),
ImprovementMarketValue,,,,-0.0000
,,,,(0.0000)


In [1629]:
PN.describe()

,AreaHA,UnburnedHA,PctRefugia,PctMasked,PctTree,PctShrub,PctHerb,PctWaterND,MEAN,Year,DOY,MS_count,ImprovementMarketValue,Z_count,BH,NH,RH,UH,UK,ZperHA,MS_perHA,A_BHperHA,A_UHperHA,ImprovPerHA,NumRefugia,RefugiaArea
count,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000,2050.00000
mean,44.38313,4.09233,9.65635,1.43250,20.88318,21.02866,47.60939,0.16725,0.01428,2000.29366,215.05707,9.14537,96242.16976,0.91171,0.22049,0.12341,0.02390,2.76732,0.13122,0.03617,0.35646,0.00556,0.13533,3281.45071,303.18049,4.09233
std,115.87187,13.19104,10.74879,4.75849,27.61168,30.21664,35.07176,1.16497,0.01164,8.78831,30.18427,64.78466,1820981.96852,15.83611,3.77350,1.45969,0.64628,17.88622,1.61854,0.44397,2.60626,0.07661,0.75736,55199.87407,818.52145,13.19105
min,4.07700,0.00630,0.02757,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1984.00000,70.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00630
25%,7.17277,0.39150,3.48222,0.00000,0.36688,0.10672,13.76872,0.00000,0.00527,1994.00000,196.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,56.00000,0.39150
50%,13.38030,0.97065,6.35939,0.00000,5.15545,1.38383,44.43235,0.00000,0.01052,2001.00000,217.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,104.00000,0.97065
75%,34.56720,2.72588,11.34183,0.00000,38.84475,38.54322,84.98155,0.02038,0.02076,2007.00000,232.00000,2.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.09336,0.00000,0.00000,0.00000,241.00000,2.72588
max,2255.49000,241.41400,91.82380,49.90010,99.34950,100.00000,100.00000,37.41810,0.05500,2014.00000,336.00000,2303.00000,71581400.00000,641.00000,122.00000,41.00000,27.00000,555.00000,37.00000,12.23846,75.30619,2.82426,11.58639,2053437.36026,18486.00000,241.41434


In [1558]:
sumstats = PN.describe().loc[['count','mean','std','min','max']].T

In [1564]:
sumstats

,count,mean,std,min,max
AreaHA,2050.00000,44.38313,115.87187,4.07700,2255.49000
UnburnedHA,2050.00000,4.09233,13.19104,0.00630,241.41400
PctRefugia,2050.00000,9.65635,10.74879,0.02757,91.82380
PctMasked,2050.00000,1.43250,4.75849,0.00000,49.90010
PctTree,2050.00000,20.88318,27.61168,0.00000,99.34950
PctShrub,2050.00000,21.02866,30.21664,0.00000,100.00000
PctHerb,2050.00000,47.60939,35.07176,0.00000,100.00000
PctWaterND,2050.00000,0.16725,1.16497,0.00000,37.41810
MEAN,2050.00000,0.01428,0.01164,0.00000,0.05500
Year,2050.00000,2000.29366,8.78831,1984.00000,2014.00000


In [1541]:
NumberRefugia_Alex = smf.ols(formula =f'NumRefugia ~ BH + UH  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1542]:
NumberRefugia_MS = smf.ols(formula =f'NumRefugia ~ MS_count  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN +  DOY + I(DOY**2) ', data=PN).fit()

In [1543]:
NumberRefugia_Z_count = smf.ols(formula =f'NumRefugia ~  Z_count  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1544]:
NumberRefugia_Z_value = smf.ols(formula =f'NumRefugia ~ ImprovementMarketValue  + AreaHA +  Year + PctTree + PctShrub + PctHerb  +  MEAN + ImprovementMarketValue  + DOY + I(DOY**2) ', data=PN).fit()

In [1545]:
summary_col([NumberRefugia_Alex, NumberRefugia_MS, NumberRefugia_Z_count, NumberRefugia_Z_value],stars=True,regressor_order=['BH','UH', 'MS_count','Z_count','ImprovementMarketValue', 'Intercept'], float_format='%0.2f',
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)})

,NumRefugia I,NumRefugia II,NumRefugia III,NumRefugia IIII
BH,1.18,,,
,(4.50),,,
UH,0.92,,,
,(0.96),,,
MS_count,,-0.07,,
,,(0.20),,
Z_count,,,-2.86***,
,,,(0.78),
ImprovementMarketValue,,,,-0.00***
,,,,(0.00)


In [1369]:
MS_ha = smf.ols(formula =f'NumRefugia ~ Year + PctTree + PctShrub + PctHerb  + MS_perHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1370]:
Z_ha = smf.ols(formula =f'NumRefugia ~ Year + PctTree + PctShrub + PctHerb  + ZperHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1372]:
ZVal_ha = smf.ols(formula =f'NumRefugia ~ Year + PctTree + PctShrub + PctHerb  +  ImprovPerHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1373]:
summary_col([A_ha, MS_ha, Z_ha, ZVal_ha],stars=True,float_format='%0.2f',
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)})

,NumRefugia I,NumRefugia II,NumRefugia III,NumRefugia IIII
AreaHA,5.18***,,,
,(0.11),,,
BH,1.18,,,
,(4.50),,,
DOY,-0.41,10.00**,10.22**,10.18**
,(2.83),(4.18),(4.17),(4.17)
I(DOY ** 2),0.00,-0.03**,-0.03***,-0.03***
,(0.01),(0.01),(0.01),(0.01)
ImprovPerHA,,,,-0.00
,,,,(0.00)


In [1174]:
A_ha = smf.ols(formula =f'PctRefugia ~ Year + PctTree + PctShrub + PctHerb  + MEAN  + A_BHperHA + A_BHperHA*MEAN + A_UHperHA  + DOY + I(DOY**2) ', data=PN).fit()

In [1175]:
MS_ha = smf.ols(formula =f'PctRefugia ~ Year + PctTree + PctShrub + PctHerb  + MEAN + MS_perHA+ MS_perHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1176]:
Z_ha = smf.ols(formula =f'PctRefugia ~ Year + PctTree + PctShrub + PctHerb  + MEAN + ZperHA + ZperHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1177]:
ZVal_ha = smf.ols(formula =f'PctRefugia ~ Year + PctTree + PctShrub + PctHerb  + MEAN + ThousImprovePerHA + ThousImprovePerHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1178]:
summary_col([A_ha, MS_ha, Z_ha, ZVal_ha],stars=True,float_format='%0.2f',
                  info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                             'R2':lambda x: "{:.2f}".format(x.rsquared)})

,PctRefugia I,PctRefugia II,PctRefugia III,PctRefugia IIII
A_BHperHA,-1.79,,,
,(3.32),,,
A_BHperHA:MEAN,-311.95,,,
,(385.05),,,
A_UHperHA,-0.29,,,
,(0.32),,,
DOY,-0.90***,-0.91***,-0.90***,-0.90***
,(0.05),(0.05),(0.05),(0.05)
I(DOY ** 2),0.00***,0.00***,0.00***,0.00***
,(0.00),(0.00),(0.00),(0.00)


In [1142]:
A_ha1 = smf.ols(formula =f'PctRefugia ~ I(AreaHA/1000) + Year + PctTree + PctShrub + PctHerb  + MEAN  + A_BHperHA + A_BHperHA*MEAN + A_UHperHA  + DOY + I(DOY**2) ', data=PN).fit()

In [1143]:
MS_ha1 = smf.ols(formula =f'PctRefugia ~ I(AreaHA/1000)+ Year + PctTree + PctShrub + PctHerb  + MEAN + MS_perHA+ MS_perHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1144]:
Z_ha1 = smf.ols(formula =f'PctRefugia ~ I(AreaHA/1000)+ Year + PctTree + PctShrub + PctHerb  + MEAN + ZperHA + ZperHA*MEAN + DOY + I(DOY**2) ', data=PN).fit()

In [1150]:
ZVal_ha1 = smf.ols(formula =f'PctRefugia ~ I(AreaHA/1000) + Year + PctTree + PctShrub + PctHerb  + MEAN + ThousImprovePerHA + DOY + I(DOY**2) ', data=PN).fit()